In [94]:
import os
import json
import requests
import pandas as pd
import numpy as np
import time 
import matplotlib.pyplot as plt

from functions import load_api_key, haversine

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
api_key = load_api_key()

In [96]:
rental_data_df = pd.read_csv('inputs/rental_with_coordinates.csv')
unique_addresses_df = rental_data_df.drop_duplicates(subset='address', keep='first')[['address', 'postal_code', 'latitude', 'longitude']]
print('No of unique addresses', rental_data_df['address'].nunique())

No of unique addresses 9542


In [97]:
station_df = pd.read_csv('inputs/stations_with_coordinates.csv')
station_df.columns = 'station_' + station_df.columns
print('No of unique stations', station_df['station_station_name'].nunique())

No of unique stations 211


In [98]:
time_distance_df = unique_addresses_df.merge(station_df, how = 'cross')
time_distance_df['walking_time_s'] = None
time_distance_df['walking_distance_m'] = None
print('No of possible combination from each address to each stations', len(time_distance_df))

No of possible combination from each address to each stations 2013362


In [99]:
time_distance_df.loc[time_distance_df['address'] == '213 CHOA CHU KANG CTRL'].head()

,address,postal_code,latitude,longitude,station_station_code,station_station_name,station_postal_code,station_latitude,station_longitude,walking_time_s,walking_distance_m
619918,213 CHOA CHU KANG CTRL,680213,1.382173,103.747212,EW1,PASIR RIS MRT STATION (EW1),519634,1.373043,103.949285,None,None
619919,213 CHOA CHU KANG CTRL,680213,1.382173,103.747212,EW2,TAMPINES MRT STATION (EW2),529538,1.353302,103.945145,None,None
619920,213 CHOA CHU KANG CTRL,680213,1.382173,103.747212,EW3,SIMEI MRT STATION (EW3),529888,1.343197,103.953377,None,None
619921,213 CHOA CHU KANG CTRL,680213,1.382173,103.747212,EW4,TANAH MERAH MRT STATION (EW4),467356,1.327187,103.946349,None,None
619922,213 CHOA CHU KANG CTRL,680213,1.382173,103.747212,EW5,BEDOK MRT STATION (EW5),467347,1.323980,103.929984,None,None


In [100]:
# resolve incorrect values
print(time_distance_df.loc[time_distance_df['postal_code'] == 'NIL', 'address'].unique())
time_distance_df.loc[time_distance_df['address'] == '215 CHOA CHU KANG CTRL', 'postal_code'] = 680215

rental_data_df.loc[rental_data_df['address'] == '215 CHOA CHU KANG CTRL', 'postal_code'] = 680215
rental_data_df.to_csv('inputs/rental_with_coordinates.csv', index = False)

[]


In [101]:
time_distance_df['station_postal_code'] = time_distance_df['station_postal_code'].astype(int)
time_distance_df['postal_code'] = time_distance_df['postal_code'].astype(int)

In [22]:
# Calculate the distance for each row
time_distance_df["hervsine_distance"] = time_distance_df.apply(
                                        lambda row: haversine(row["latitude"], row["longitude"], row["station_latitude"], row["station_longitude"]), axis=1)

In [23]:
time_distance_df

,address,postal_code,latitude,longitude,station_station_code,station_station_name,station_postal_code,station_latitude,station_longitude,walking_time_s,walking_distance_m,hervsine_distance
0,105 ANG MO KIO AVE 4,560105,1.372313,103.837601,EW1,PASIR RIS MRT STATION (EW1),519634,1.373043,103.949285,None,None,12.415331
1,105 ANG MO KIO AVE 4,560105,1.372313,103.837601,EW2,TAMPINES MRT STATION (EW2),529538,1.353302,103.945145,None,None,12.140377
2,105 ANG MO KIO AVE 4,560105,1.372313,103.837601,EW3,SIMEI MRT STATION (EW3),529888,1.343197,103.953377,None,None,13.271073
3,105 ANG MO KIO AVE 4,560105,1.372313,103.837601,EW4,TANAH MERAH MRT STATION (EW4),467356,1.327187,103.946349,None,None,13.088833
4,105 ANG MO KIO AVE 4,560105,1.372313,103.837601,EW5,BEDOK MRT STATION (EW5),467347,1.323980,103.929984,None,None,11.591010
...,...,...,...,...,...,...,...,...,...,...,...,...
2013357,272 BANGKIT RD,670272,1.377573,103.774462,PW3,PUNGGOL POINT LRT STATION (PW3),828644,1.416849,103.906651,None,None,15.329590
2013358,272 BANGKIT RD,670272,1.377573,103.774462,PW4,SAMUDERA LRT STATION (PW4),828645,1.415902,103.902156,None,None,14.820747
2013359,272 BANGKIT RD,670272,1.377573,103.774462,PW5,NIBONG LRT STATION (PW5),828862,1.411854,103.900339,None,None,14.502706
2013360,272 BANGKIT RD,670272,1.377573,103.774462,PW6,SUMANG LRT STATION (PW6),828863,1.408452,103.898558,None,None,14.215711


In [24]:
# Group by 'address' and get the top 3 smallest distances
filtered_time_distance_df = (
    time_distance_df.groupby("address", group_keys=False)
    .apply(lambda group: group.nsmallest(3, "hervsine_distance"), include_groups=False))

filtered_time_distance_df = filtered_time_distance_df.reset_index(drop = True)
filtered_time_distance_df

,postal_code,latitude,longitude,station_station_code,station_station_name,station_postal_code,station_latitude,station_longitude,walking_time_s,walking_distance_m,hervsine_distance
0,190001,1.303671,103.864479,CC5,NICOLL HIGHWAY MRT STATION (CC5),38970,1.299767,103.863637,None,None,0.444129
1,190001,1.303671,103.864479,EW11,LAVENDER MRT STATION (EW11),208699,1.307378,103.862768,None,None,0.453915
2,190001,1.303671,103.864479,DT14,BUGIS MRT STATION (DT14),188476,1.299314,103.857126,None,None,0.950214
3,460001,1.320852,103.933721,EW5,BEDOK MRT STATION (EW5),467347,1.323980,103.929984,None,None,0.541764
4,460001,1.320852,103.933721,TE28,SIGLAP MRT STATION (TE28),449969,1.310009,103.930026,None,None,1.273786
...,...,...,...,...,...,...,...,...,...,...,...
28621,162009,1.286827,103.828659,TE16,HAVELOCK MRT STATION (TE16),168604,1.288505,103.833586,None,None,0.578563
28622,162009,1.286827,103.828659,TE15,GREAT WORLD MRT STATION (TE15),238384,1.293948,103.833721,None,None,0.971369
28623,163009,1.286814,103.828928,EW17,TIONG BAHRU MRT STATION (EW17),168731,1.286103,103.827445,None,None,0.182888
28624,163009,1.286814,103.828928,TE16,HAVELOCK MRT STATION (TE16),168604,1.288505,103.833586,None,None,0.550850


In [53]:
filtered_time_distance_df['postal_code'].nunique()

9541

In [13]:
start = time.time()

for row in range(len(filtered_time_distance_df)):
    start_latitude  = float(filtered_time_distance_df.loc[row, 'latitude'])
    start_longitude = float(filtered_time_distance_df.loc[row, 'longitude'])    
    end_latitude    = float(filtered_time_distance_df.loc[row, 'station_latitude'])
    end_longitude   = float(filtered_time_distance_df.loc[row, 'station_longitude'])   

    url = f"https://www.onemap.gov.sg/api/public/routingsvc/route?start={start_latitude}%2C{start_longitude}&end={end_latitude}%2C{end_longitude}&routeType=walk"
    headers = {"Authorization": api_key}
    response = requests.request("GET", url, headers=headers)
    while response.status_code != 200:
        time.sleep(169)
        response = requests.request("GET", url, headers=headers)
    parsed_data = json.loads(response.text)    
    filtered_time_distance_df.loc[row, 'walking_time_s'] = parsed_data['route_summary']['total_time'] 
    filtered_time_distance_df.loc[row, 'walking_distance_m'] = parsed_data['route_summary']['total_distance'] 

    if row%100 == 0:
        print(row, (time.time() - start)/60)

0 0.009898177782694499



KeyboardInterrupt



In [31]:
filtered_time_distance_df

,postal_code,latitude,longitude,station_station_code,station_station_name,station_postal_code,station_latitude,station_longitude,walking_time_s,walking_distance_m,hervsine_distance
0,190001,1.303671,103.864479,CC5,NICOLL HIGHWAY MRT STATION (CC5),38970,1.299767,103.863637,438,609,0.444129
1,190001,1.303671,103.864479,EW11,LAVENDER MRT STATION (EW11),208699,1.307378,103.862768,512,711,0.453915
2,190001,1.303671,103.864479,DT14,BUGIS MRT STATION (DT14),188476,1.299314,103.857126,787,1093,0.950214
3,460001,1.320852,103.933721,EW5,BEDOK MRT STATION (EW5),467347,1.323980,103.929984,690,959,0.541764
4,460001,1.320852,103.933721,TE28,SIGLAP MRT STATION (TE28),449969,1.310009,103.930026,1191,1654,1.273786
...,...,...,...,...,...,...,...,...,...,...,...
28621,162009,1.286827,103.828659,TE16,HAVELOCK MRT STATION (TE16),168604,1.288505,103.833586,569,790,0.578563
28622,162009,1.286827,103.828659,TE15,GREAT WORLD MRT STATION (TE15),238384,1.293948,103.833721,1139,1581,0.971369
28623,163009,1.286814,103.828928,EW17,TIONG BAHRU MRT STATION (EW17),168731,1.286103,103.827445,199,276,0.182888
28624,163009,1.286814,103.828928,TE16,HAVELOCK MRT STATION (TE16),168604,1.288505,103.833586,585,812,0.550850


# Handle previous wrong postal code

In [54]:
filtered_time_distance_df = pd.read_csv('inputs/travelling_distance.csv')

In [81]:
df = pd.DataFrame({'postal_code' : [530021],
                  'latitude' : [1.364246],
                  'longitude' : [103.8914777],
                  })

In [82]:
special_df = df.merge(station_df, how = 'cross')

In [83]:
special_df["hervsine_distance"] = special_df.apply(
                                        lambda row: haversine(row["latitude"], row["longitude"], row["station_latitude"], row["station_longitude"]), axis=1)

In [84]:
special_df

,postal_code,latitude,longitude,station_station_code,station_station_name,station_postal_code,station_latitude,station_longitude,hervsine_distance
0,530021,1.364246,103.891478,EW1,PASIR RIS MRT STATION (EW1),519634,1.373043,103.949285,6.500022
1,530021,1.364246,103.891478,EW2,TAMPINES MRT STATION (EW2),529538,1.353302,103.945145,6.088664
2,530021,1.364246,103.891478,EW3,SIMEI MRT STATION (EW3),529888,1.343197,103.953377,7.268159
3,530021,1.364246,103.891478,EW4,TANAH MERAH MRT STATION (EW4),467356,1.327187,103.946349,7.361157
4,530021,1.364246,103.891478,EW5,BEDOK MRT STATION (EW5),467347,1.323980,103.929984,6.194376
...,...,...,...,...,...,...,...,...,...
206,530021,1.364246,103.891478,PW3,PUNGGOL POINT LRT STATION (PW3),828644,1.416849,103.906651,6.087465
207,530021,1.364246,103.891478,PW4,SAMUDERA LRT STATION (PW4),828645,1.415902,103.902156,5.865234
208,530021,1.364246,103.891478,PW5,NIBONG LRT STATION (PW5),828862,1.411854,103.900339,5.384696
209,530021,1.364246,103.891478,PW6,SUMANG LRT STATION (PW6),828863,1.408452,103.898558,4.978150


In [87]:
filtered_special_df = special_df.sort_values('hervsine_distance', ascending = True).iloc[:3]
filtered_special_df = filtered_special_df.reset_index(drop = True)
filtered_special_df

,postal_code,latitude,longitude,station_station_code,station_station_name,station_postal_code,station_latitude,station_longitude,hervsine_distance
0,530021,1.364246,103.891478,NE14,HOUGANG MRT STATION (NE14),538758,1.371292,103.892381,0.789910
1,530021,1.364246,103.891478,NE13,KOVAN MRT STATION (NE13),534799,1.360179,103.885065,0.844206
2,530021,1.364246,103.891478,NE15,BUANGKOK MRT STATION (NE15),545061,1.382870,103.893123,2.078953


In [88]:
start = time.time()

for row in range(len(filtered_special_df)):
    start_latitude  = float(filtered_special_df.loc[row, 'latitude'])
    start_longitude = float(filtered_special_df.loc[row, 'longitude'])    
    end_latitude    = float(filtered_special_df.loc[row, 'station_latitude'])
    end_longitude   = float(filtered_special_df.loc[row, 'station_longitude'])   

    url = f"https://www.onemap.gov.sg/api/public/routingsvc/route?start={start_latitude}%2C{start_longitude}&end={end_latitude}%2C{end_longitude}&routeType=walk"
    headers = {"Authorization": api_key}
    response = requests.request("GET", url, headers=headers)
    while response.status_code != 200:
        time.sleep(169)
        response = requests.request("GET", url, headers=headers)
    parsed_data = json.loads(response.text)    
    filtered_special_df.loc[row, 'walking_time_s'] = parsed_data['route_summary']['total_time'] 
    filtered_special_df.loc[row, 'walking_distance_m'] = parsed_data['route_summary']['total_distance'] 


In [89]:
filtered_special_df

,postal_code,latitude,longitude,station_station_code,station_station_name,station_postal_code,station_latitude,station_longitude,hervsine_distance,walking_time_s,walking_distance_m
0,530021,1.364246,103.891478,NE14,HOUGANG MRT STATION (NE14),538758,1.371292,103.892381,0.789910,844.0,1172.0
1,530021,1.364246,103.891478,NE13,KOVAN MRT STATION (NE13),534799,1.360179,103.885065,0.844206,726.0,1009.0
2,530021,1.364246,103.891478,NE15,BUANGKOK MRT STATION (NE15),545061,1.382870,103.893123,2.078953,1896.0,2632.0


In [91]:
filtered_time_distance_df = pd.concat([filtered_time_distance_df, filtered_special_df], axis = 0).reset_index(drop = True)

In [92]:
filtered_time_distance_df.to_csv('inputs/travelling_distance.csv', index = False)